In [2]:
import numpy as n
import pandas as p
import matplotlib.pyplot as m
d= p.read_csv("hotel_experience.csv")
d.head()

,Unnamed: 0,id,Gender,Age,purpose_of_travel,Type of Travel,Type Of Booking,Hotel wifi service,Departure/Arrival convenience,Ease of Online booking,Hotel location,Food and drink,Stay comfort,Common Room entertainment,Checkin/Checkout service,Other service,Cleanliness,overall experience
0,0,32746,Female,32,academic,Group Travel,Individual/Couple,5,5,2,5,5,5,5,1,2,5,good
1,1,52532,Male,44,business,Group Travel,Group bookings,3,3,3,3,5,3,4,2,4,1,good
2,2,62644,Male,64,tourism,Personal Travel,Individual/Couple,3,5,3,3,1,1,1,3,5,1,bad
3,3,57270,Female,36,academic,Group Travel,Group bookings,2,4,4,4,4,3,2,1,2,2,bad
4,4,18607,Female,80,tourism,Group Travel,Individual/Couple,2,4,2,5,4,4,4,2,2,4,bad


In [3]:
from sklearn import preprocessing
l= preprocessing.LabelEncoder()
d["Gender_n"]=l.fit_transform(d["Gender"])                           #Female:0 male: 1 
d["purpose_of_travel_n"]=l.fit_transform(d["purpose_of_travel"])     #academic:0 aviation:1 business:2 personal:3 tourism:4 
d["Type of Travel_n"]=l.fit_transform(d["Type of Travel"])           #Group Travel:0 Personal Travel:1
d["Type Of Booking_n"]=l.fit_transform(d["Type Of Booking"])         #Group bookings:0 Individual/Couple:1 
d["overall experience_n"]=l.fit_transform(d["overall experience"])   #bad:0 good:1
d1=d.drop(["Gender","purpose_of_travel","Type of Travel","Type Of Booking","overall experience"], axis='columns')
d1.head()

,Unnamed: 0,id,Age,Hotel wifi service,Departure/Arrival convenience,Ease of Online booking,Hotel location,Food and drink,Stay comfort,Common Room entertainment,Checkin/Checkout service,Other service,Cleanliness,Gender_n,purpose_of_travel_n,Type of Travel_n,Type Of Booking_n,overall experience_n
0,0,32746,32,5,5,2,5,5,5,5,1,2,5,0,0,0,1,1
1,1,52532,44,3,3,3,3,5,3,4,2,4,1,1,2,0,0,1
2,2,62644,64,3,5,3,3,1,1,1,3,5,1,1,4,1,1,0
3,3,57270,36,2,4,4,4,4,3,2,1,2,2,0,0,0,0,0
4,4,18607,80,2,4,2,5,4,4,4,2,2,4,0,4,0,1,0


In [4]:
d_main=d1.drop(['Unnamed: 0','id'], axis='columns')
d_main.head()

,Age,Hotel wifi service,Departure/Arrival convenience,Ease of Online booking,Hotel location,Food and drink,Stay comfort,Common Room entertainment,Checkin/Checkout service,Other service,Cleanliness,Gender_n,purpose_of_travel_n,Type of Travel_n,Type Of Booking_n,overall experience_n
0,32,5,5,2,5,5,5,5,1,2,5,0,0,0,1,1
1,44,3,3,3,3,5,3,4,2,4,1,1,2,0,0,1
2,64,3,5,3,3,1,1,1,3,5,1,1,4,1,1,0
3,36,2,4,4,4,4,3,2,1,2,2,0,0,0,0,0
4,80,2,4,2,5,4,4,4,2,2,4,0,4,0,1,0


In [124]:
#training_data
from sklearn.model_selection import train_test_split
X=d_main.drop(['Age'],axis='columns')
X_train,X_test = train_test_split(X,test_size = 0.2,random_state=1)

In [127]:
def gini(dataset,target):
    groups,group_count=n.unique(dataset[target],return_counts=True)
    gini=1
    for i in range(len(groups)):
        p=group_count[i]/sum(group_count)
        gini-=p**2
    return gini
def ig(dataset,feature,target):
    initial_gini=gini(dataset,target)
    ig=initial_gini
    value,value_count=n.unique(dataset[feature],return_counts=True)
    weighted_feature_entropy = n.sum([(value_count[i]/n.sum(value_count))*gini(dataset.where(dataset[feature]
                              ==value[i]).dropna(),target) for i in range(len(value))])    
    gain = initial_gini - weighted_feature_entropy
    return gain
def tree(n_dataset,dataset,features,target,root):
    value,value_count=n.unique(dataset[target],return_counts=True)
    if len(value)<=1:
        return value
    elif len(dataset)==0:
        return value[n.argmax(value_count)]
    elif len(features)==0:
        return root
    elif gini(dataset,target)==0:
        return n.unique(dataset[target],return_counts=False)
    else:
        root=value[n.argmax(value_count)]
        item_values= [ig(dataset,feature,target) for feature in features]
        optimum_feature=features[n.argmax(item_values)]
        decision_tree={optimum_feature:{}}
        features=[i for i in features if i!= optimum_feature]